In [4]:
import matplotlib.pyplot as plt
import numpy as np

from pypokerengine.api.game import setup_config, start_poker

In [5]:
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate
import inspect

NB_SIMULATION = 100

class HonestPlayer(BasePokerPlayer):

    def declare_action(self, valid_actions, hole_card, round_state):
        curframe = inspect.currentframe()
        calframe = inspect.getouterframes(curframe, 2)
        #print('caller name:', calframe[1][3])
        community_card = round_state['community_card']
        win_rate = estimate_hole_card_win_rate(
                nb_simulation=NB_SIMULATION,
                nb_player=self.nb_player,
                hole_card=gen_cards(hole_card),
                community_card=gen_cards(community_card)
                )
        if win_rate >= 0.5 / self.nb_player:
            action = valid_actions[1]  # fetch CALL action info
        else:
            action = valid_actions[0]  # fetch FOLD action info
        return action['action'], action['amount']

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass



In [6]:
results_dict = {'p1': [], 'p2': [], 'p3': []}
initial_stack = 1000
for i in range(100):
    config = setup_config(max_round=10, initial_stack=initial_stack, small_blind_amount=5)
    config.register_player(name="p1", algorithm=HonestPlayer())
    config.register_player(name="p2", algorithm=HonestPlayer())
    config.register_player(name="p3", algorithm=HonestPlayer())
    game_result = start_poker(config, verbose=0)
    print(i, game_result)
    players = game_result['players']
    for player in players:
        results_dict[player['name']] += [player['stack'] - initial_stack]

#print(results_dict)



caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond_to_ask
caller name: respond

KeyboardInterrupt: 

In [ ]:

def plot_running_sums(data):
  """Plots the running sums of a dictionary of arrays.

  Args:
    data: A dictionary where the keys are labels and the values are arrays of numbers.
  """

  for label, values in data.items():
    running_sum = np.cumsum(values)
    plt.plot(running_sum, label=label)

  plt.xlabel("Index")
  plt.ylabel("Running Sum")
  plt.legend()
  plt.show()

plot_running_sums(results_dict)

In [ ]:
from pypokerengine.players import BasePokerPlayer

class DummyPlayer(BasePokerPlayer):
    def __init__(self):
        super().__init__()
        self.initial_game_info = None
        #print('Init: ', self.name)

    def declare_action(self, valid_actions, hole_card, round_state):
        print("valid_actions", valid_actions)
        print('round state', round_state)
        action = valid_actions[0]
        return action['action'], action['amount']

    def receive_game_start_message(self, game_info):
        print('receive_game_start_message', game_info)
        self.initial_game_info = game_info.copy()
        
    def receive_round_start_message(self, round_count, hole_card, seats):
        print('receive_round_start_message')
        pass

    def receive_street_start_message(self, street, round_state):
        print('receive_street_start_message')
        pass
        
    def receive_game_update_message(self, action, round_state):
        print('receive_game_update_message')
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        print('receive_round_result_message')
        pass

config = setup_config(max_round=1, initial_stack=initial_stack, small_blind_amount=5)
config.register_player(name="p1", algorithm=DummyPlayer())
config.register_player(name="p2", algorithm=FishPlayer())
config.register_player(name="p3", algorithm=HonestPlayer())
game_result = start_poker(config, verbose=0)

In [ ]:
def exp_val(pot, bet, p_win):
    return p_win * (pot - bet) + (1-p_win) * (- bet)

samples = 100
pots = np.linspace(1, 100, samples)
bets = np.linspace(1, 100, samples)
p_wins = np.linspace(0, 1, samples)

In [ ]:
exp_vals = np.zeros((samples,samples,samples))

for i,pot in enumerate(pots):
    for j,bet in enumerate(bets):
        for k,p_win in enumerate(p_wins):
                   exp_vals[i,j,k] = exp_val(pot, bet, p_win)
            

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the figure and 3D axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Generate data for plotting
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
x, y = np.meshgrid(x, y)
z = np.sin(np.sqrt(x**2 + y**2))

# Create the 3D surface plot
ax.plot_surface(x, y, z, cmap='viridis')

# Labels
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')

# Show the plot
plt.show()
